In [193]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
import statsmodels .api as sm 
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
sns.set()


In [194]:
synthetic_training_set_path = "Model data/train.csv"

Real_World_training_set_path = 'Real World Data/healthcare-dataset-stroke-data.csv'


synthetic_training_set = pd.read_csv(synthetic_training_set_path)

Real_World_training_set = pd.read_csv(Real_World_training_set_path)


In [195]:
train_set= pd.concat([ synthetic_training_set, Real_World_training_set])
train_set

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [196]:
train_set.info() 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20414 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 20414 non-null  int64  
 1   gender             20414 non-null  object 
 2   age                20414 non-null  float64
 3   hypertension       20414 non-null  int64  
 4   heart_disease      20414 non-null  int64  
 5   ever_married       20414 non-null  object 
 6   work_type          20414 non-null  object 
 7   Residence_type     20414 non-null  object 
 8   avg_glucose_level  20414 non-null  float64
 9   bmi                20213 non-null  float64
 10  smoking_status     20414 non-null  object 
 11  stroke             20414 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


In [197]:
train_set.describe(include='all')

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
count,20414.000000,20414,20414.000000,20414.000000,20414.000000,20414,20414,20414,20414.000000,20213.000000,20414,20414.000000
unique,NaN,3,NaN,NaN,NaN,2,5,2,NaN,NaN,4,NaN
top,NaN,Female,NaN,NaN,NaN,Yes,Private,Urban,NaN,NaN,never smoked,NaN
freq,NaN,12440,NaN,NaN,NaN,13738,12677,10236,NaN,NaN,8173,NaN
mean,14877.273636,NaN,41.870510,0.061673,0.031008,NaN,NaN,NaN,93.322256,28.302280,NaN,0.043157
std,16825.306948,NaN,21.756482,0.240567,0.173344,NaN,NaN,NaN,32.476351,7.021765,NaN,0.203215
min,0.000000,NaN,0.080000,0.000000,0.000000,NaN,NaN,NaN,55.120000,10.300000,NaN,0.000000
25%,4766.250000,NaN,25.000000,0.000000,0.000000,NaN,NaN,NaN,75.220000,23.500000,NaN,0.000000
50%,9511.500000,NaN,43.000000,0.000000,0.000000,NaN,NaN,NaN,86.250000,27.700000,NaN,0.000000
75%,14279.750000,NaN,58.000000,0.000000,0.000000,NaN,NaN,NaN,99.730000,32.200000,NaN,0.000000


In [198]:
data = train_set.copy()
data['gender'] =data['gender'].map({'Female':0,'Male':1,'Other':2})
data['work_type'] = data['work_type'].map({'Private':4,'Self-employed':3,'Govt_job':1,'children':2,'Never_worked':0})
data['ever_married'] = data['ever_married'].map({'No':0,'Yes':1})
data['smoking_status']= data['smoking_status'].map({'formerly smoked':2 ,'never smoked':0 , 'smokes':4 , 'Unknown':1})
data['Residence_type'] = data["Residence_type"].map({'Urban':1 , 'Rural':0})

data.dtypes


id                     int64
gender                 int64
age                  float64
hypertension           int64
heart_disease          int64
ever_married           int64
work_type              int64
Residence_type         int64
avg_glucose_level    float64
bmi                  float64
smoking_status         int64
stroke                 int64
dtype: object

In [199]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20414 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 20414 non-null  int64  
 1   gender             20414 non-null  int64  
 2   age                20414 non-null  float64
 3   hypertension       20414 non-null  int64  
 4   heart_disease      20414 non-null  int64  
 5   ever_married       20414 non-null  int64  
 6   work_type          20414 non-null  int64  
 7   Residence_type     20414 non-null  int64  
 8   avg_glucose_level  20414 non-null  float64
 9   bmi                20213 non-null  float64
 10  smoking_status     20414 non-null  int64  
 11  stroke             20414 non-null  int64  
dtypes: float64(3), int64(9)
memory usage: 2.0 MB


In [200]:
bmi_mean =data['bmi'].mean()
print(f"bmi mean is : {bmi_mean}")

bmi_median =data['bmi'].median()
print(f"bmi median is : {bmi_median}")
data['bmi'].fillna(data['bmi'].median(),inplace =True)
data.head()

bmi mean is : 28.30227972097165
bmi median is : 27.7


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,1,28.0,0,0,1,4,1,79.53,31.1,0,0
1,1,1,33.0,0,0,1,4,0,78.44,23.9,2,0
2,2,0,42.0,0,0,1,4,0,103.00,40.3,1,0
3,3,1,56.0,0,0,1,4,1,64.87,28.8,0,0
4,4,0,24.0,0,0,0,4,0,73.36,28.8,0,0


In [201]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20414 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 20414 non-null  int64  
 1   gender             20414 non-null  int64  
 2   age                20414 non-null  float64
 3   hypertension       20414 non-null  int64  
 4   heart_disease      20414 non-null  int64  
 5   ever_married       20414 non-null  int64  
 6   work_type          20414 non-null  int64  
 7   Residence_type     20414 non-null  int64  
 8   avg_glucose_level  20414 non-null  float64
 9   bmi                20414 non-null  float64
 10  smoking_status     20414 non-null  int64  
 11  stroke             20414 non-null  int64  
dtypes: float64(3), int64(9)
memory usage: 2.0 MB


In [202]:
data.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [203]:
est = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married','work_type', 'Residence_type', 'avg_glucose_level', 'bmi','smoking_status']
y = data['stroke']
x1 = data[est]

In [204]:
x = sm.add_constant(x1)
log_reg = sm.Logit(y, x)
results = log_reg.fit()
results.summary()


Optimization terminated successfully.
         Current function value: 0.134265
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 stroke   No. Observations:                20414
Model:                          Logit   Df Residuals:                    20403
Method:                           MLE   Df Model:                           10
Date:                Thu, 11 May 2023   Pseudo R-squ.:                  0.2451
Time:                        22:08:09   Log-Likelihood:                -2740.9
converged:                       True   LL-Null:                       -3630.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -9.2382      0.341    -27.116      0.000      -9.906      -8.571
gender                0.0711      0.076      0.937      0.349      -0.078       0.220
age                   0.0846      0.003     28.570      0.000       0.079       0.090
hypertension          0.4162      0.095      4.365      0.000       0.229       0.603
heart_disease         0.1424      0.121      1.181      0.238      -0.094       0.379
ever_married         -0.1393      0.138     -1.006      0.314      -0.411       0.132
work_type             0.0562      0.040      1.416      0.157      -0.022       0.134
Residence_type       -0.0145      0.074     -0.197      0.844      -0.159       0.130
avg_glucose_level     0.0057      0.001      7.637      0.000       0.004       0.007
bmi                   0.0159      0.006      2.506      0.012       0.003       0.028
smoking_status        0.0918      0.027      3.444      0.001       0.040       0.144
=====================================================================================
"""

In [205]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
train_X, val_X, train_y, val_y = train_test_split(x1, y, test_size = 0.2)
model = LogisticRegression()
model.fit(x1, y)

val_predictions = model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

confusion_matrix(val_y, val_predictions)


0.04335047759000735


c:\Users\yaman\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[3903,    3],
       [ 174,    3]], dtype=int64)

In [206]:
accuracy = accuracy_score(val_y, val_predictions)
accuracy

0.9566495224099927

In [207]:
test_set = pd.read_csv('test.csv')

In [208]:
test_data = test_set.copy()
test_data['gender'] =test_data['gender'].map({'Female':0,'Male':1,'Other':2})
test_data['work_type'] = test_data['work_type'].map({'Private':4,'Self-employed':3,'Govt_job':1,'children':2,'Never_worked':0})
test_data['ever_married'] = test_data['ever_married'].map({'No':0,'Yes':1})
test_data['smoking_status']= test_data['smoking_status'].map({'formerly smoked':2 ,'never smoked':0 , 'smokes':4 , 'Unknown':1})
test_data['Residence_type'] = test_data["Residence_type"].map({'Urban':1 , 'Rural':0})


In [209]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10204 entries, 0 to 10203
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10204 non-null  int64  
 1   gender             10204 non-null  int64  
 2   age                10204 non-null  float64
 3   hypertension       10204 non-null  int64  
 4   heart_disease      10204 non-null  int64  
 5   ever_married       10204 non-null  int64  
 6   work_type          10204 non-null  int64  
 7   Residence_type     10204 non-null  int64  
 8   avg_glucose_level  10204 non-null  float64
 9   bmi                10204 non-null  float64
 10  smoking_status     10204 non-null  int64  
dtypes: float64(3), int64(8)
memory usage: 877.0 KB


In [210]:
test_X = test_data[est]
preds = model.predict(test_X)

output = pd.DataFrame({'id': test_data['id'], 'stroke': preds})

In [211]:
output.to_csv('sub.csv',index=False)